In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/Data Augmentation-XrayImg

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/Data Augmentation-XrayImg


#0. Setup

In [ ]:
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/{cu_version}/{torch_version}/index.html

Looking in links: https://download.openmmlab.com/mmcv/dist/{cu_version}/{torch_version}/index.html
     |████████████████████████████████| 256kB 11.0MB/s 
     |████████████████████████████████| 194kB 21.2MB/s 


In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -r requirementsa/build.txt
!pip install -v -e .
!mkdir checkpoints/ 

# 1. Import Libraries

In [ ]:
import torch
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torch.utils.data import Dataset
import os
import glob
import cv2
import matplotlib.pyplot as plt
import json
from tqdm import tqdm
import mmcv
from mmdet.apis import init_detector, inference_detector
from matplotlib import patches
import numpy as np 
from torchvision import transforms


# 1.5 Segmentation

In [ ]:
train_annotations_path='../XRAYBoneDataset/annotations/instances_train.json'
val_annotations_path='../XRAYBoneDataset/annotations/instances_val.json'
test_annotations_path='../XRAYBoneDataset/annotations/instances_test.json'

train_images_path='../XRAYBoneDataset/train/JPEGImages'
val_images_path='../XRAYBoneDataset/val/JPEGImages'
test_images_path='../XRAYBoneDataset/test/JPEGImages'

In [ ]:
def read_json(path):
  with open(path,'r') as f:
    data = json.load(f)
  return data


def write_json(dic,path):
  with open(path,'w') as f:
    json.dump(dic,f,indent=4)

In [ ]:
data=read_json(train_annotations_path)
print(data['categories'])

In [ ]:
!wget -P ./checkpoints/ http://download.openmmlab.com/mmdetection/v2.0/mask_rcnn/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco/mask_rcnn_r50_caffe_fpn_mstrain-poly_3x_coco_bbox_mAP-0.408__segm_mAP-0.37_20200504_163245-42aa3d00.pth

In [ ]:
# TRAIN Seg
!python3 ./tools/train.py ../train_xray_segm.py

# 2. Configs

In [ ]:
data_train_path = '../original/dataDetect_train'
data_test_path = '../original/dataDetect_test'
train_data_json='../original/train_data.json'
test_data_json='../original/test_data.json'
config_file = '../train_xray_segm.py'
checkpoint_file = './work_dirs/train_xray_segm/latest.pth'
device='cuda' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE=4 # CUDA out of memory :(
EPOCH_N=5
CLASS_N=2
WIDTH=300
HEIGHT=300

# 3. Utils

In [ ]:
def get_anno(txt_path):
    annos=[]
    with open(txt_path) as f:
      for line in f.readlines():
        anno=line[:-1].split(" ")
        anno = [float(num) for num in anno]
        annos.append(anno[1:])
    return annos


def get_offset(annos,image): # for single image
  new_annos=[]
  for annotation in annos:
    bbox = [annotation[0] * image.shape[1], annotation[1] * image.shape[0], 
            annotation[2] * image.shape[1], annotation[3] * image.shape[0]]
    new_annos.append(bbox)
  return new_annos


def get_path(data_path):
  img_paths = glob.glob(os.path.join(data_path,"*.png"))
  dic={}
  for img_path in img_paths:
    txt_path = img_path.replace('.png','.txt')
    dic[img_path]=get_anno(txt_path)
  return dic


def my_draw(img_path,anno):
  image=cv2.imread(img_path)
  print(image.shape)
  bboxs=get_offset(anno,image)
  for bbox in bboxs:
    cv2.rectangle(image, (int(bbox[0] - bbox[2] / 2), int(bbox[1] - bbox[3] / 2)), (int(bbox[0] + bbox[2] / 2), int(bbox[1] + bbox[3] / 2)), (225, 0, 0), 2)
  plt.figure(figsize = (10, 10))
  plt.imshow(image)
  plt.show()


def convert_bin(result):
  x=result[1][0][0]
  new_result=[]
  for i in range(len(x)):
    new_x=[]
    for j in range(len(x[i])):
      if x[i][j]:
        new_x.append(1)
      else:
        new_x.append(0)
    new_result.append(new_x)
  return new_result # mask
      

def remove_background(img,mask):
  
  img=np.transpose(img, (2, 0, 1))
  for channel in img:
    for i in range(len(channel)):
      for j in range(len(channel[i])):
        if mask[i][j]==0:
          channel[i][j]=0
  
  return np.transpose(img, (1, 2, 0))


def save_new_img(original_img_paths,save_img_folder):
  image_paths = glob.glob(os.path.join(original_img_paths,'*.jpg'))
  for i,img_path in enumerate(image_paths):
    img=cv2.imread(img_path)
    result=inference_detector(model, img_path)
    new_re=convert_bin(result)
    new_re=np.array(new_re)
    new_re=remove_background(img,new_re)
    clahe_img=equalize_clahe_color(new_re)
    save_img_path=os.path.join(save_img_folder,'%d.jpg'%i)
    cv2.imwrite(save_img_path,clahe_img)


def equalize_clahe_color(img):
    cla = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(18, 18))
    channels = cv2.split(img)
    eq_channels = []
    for ch in channels:
        eq_channels.append(cla.apply(ch))
    eq_image = cv2.merge(eq_channels)
    return eq_image



In [ ]:
# test one sample
test_dic=read_json(test_data_json)
item=next(iter(test_dic.items()))
my_draw(item[0],item[1])

In [ ]:
# demo img
!ls ../

seg_model = init_detector(config_file, checkpoint_file, device=device)
path = item[0]
img=cv2.imread(path)
result=inference_detector(seg_model.cpu(), path)
new_re=convert_bin(result)
new_re=np.array(new_re)
new_re=remove_background(img,new_re)
new_re=equalize_clahe_color(new_re)
plt.imshow(new_re)
plt.show()


# 4. Dataset

In [ ]:

class XrayImgDataset(Dataset):
  def __init__(self,json_file,phase):
    self.data=read_json(json_file)
    self.img_paths=list(self.data.keys())
    self.bbxs=list(self.data.values())
    self.phase=phase
    self.transforms=torchvision.transforms.ToTensor()

  def convert_offsets(self,bbx,img):
    bboxs=get_offset(bbx,img)
    new_bbx=[[int(bbox[0] - bbox[2] / 2), int(bbox[1] - bbox[3] / 2),int(bbox[0] + bbox[2] / 2),int(bbox[1] + bbox[3] / 2)] for bbox in bboxs]
    return new_bbx

    
  def __getitem__(self,idx):
    img_path=self.img_paths[idx]
    img=cv2.imread(img_path)
    result=inference_detector(seg_model.cpu(), img_path)
    new_re=convert_bin(result)
    new_re=np.array(new_re)
    new_re=remove_background(img,new_re)
    img=equalize_clahe_color(new_re)
    bbxs=self.convert_offsets(self.bbxs[idx],img)
    img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # RGB
    
    for bbx in bbxs:
      bbx[0]=bbx[0]/img.shape[1]*WIDTH
      bbx[1]=bbx[1]/img.shape[0]*HEIGHT
      bbx[2]=bbx[2]/img.shape[1]*WIDTH
      bbx[3]=bbx[3]/img.shape[0]*HEIGHT
    img=cv2.resize(img,(WIDTH,HEIGHT))
    img=img.astype(float)
    img=self.transforms(img)
    img=img/255

    bbxs=torch.FloatTensor(bbxs)
    targets={}
    targets['boxes']=bbxs
    targets['labels']=torch.ones(bbxs.shape[0]).long()
    return img,targets

  def __len__(self):
    return len(self.data)

In [ ]:
train_data=XrayImgDataset(train_data_json,'train')
train_dataloader = torch.utils.data.DataLoader(
   train_data,
   batch_size=BATCH_SIZE,
   num_workers=2,
   shuffle=True,
   collate_fn=lambda x: list(zip(*x)),
   )


val_data=XrayImgDataset(test_data_json,'val')
val_dataloader = torch.utils.data.DataLoader(
   val_data,
   batch_size=2,
   num_workers=2,
   shuffle=True,
   collate_fn=lambda x: list(zip(*x)),
   )


In [ ]:
# test one
item=iter(train_dataloader).next()
print(item[0])

# 5. Model

In [ ]:
model=torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
in_features=model.roi_heads.box_predictor.cls_score.in_features
print(in_features)
model.roi_heads.box_predictor=FastRCNNPredictor(in_features,CLASS_N)
model=model.to(device)
params=[p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4, weight_decay=5e-4)
#print(list(model.children()))

In [ ]:
best_point=9999


# 6. Train

In [ ]:
model.load_state_dict(torch.load("../end2end.pth"))

for epoch in range(EPOCH_N):
	print("Training...")
	train_loss=0
	val_loss=0
	for imgs,targets in tqdm(train_dataloader):
		model.train()
		model=model.double()
		optimizer.zero_grad()
		imgs=list(img.to(device) for img in imgs)
		targets=[{k:v.to(device) for k,v in t.items()} for t in targets]
		loss_dict=model(imgs,targets)
		losses=sum(loss for loss in loss_dict.values())   	
		losses.backward()
		optimizer.step()
		train_loss+=losses.item()

	for imgs,targets in tqdm(val_dataloader):
		model.eval()
		imgs=list(img.to(device) for img in imgs)
		targets=[{k:v.to(device) for k,v in t.items()} for t in targets]
		loss_dict_val=model(imgs,targets)
		losses_val=sum(loss for loss in loss_dict.values())
		val_loss+=losses_val.item()
		if losses_val.item()<best_point:
			best_point=losses_val.item()
			torch.save(model.state_dict(), "../end2end.pth")

		#print(losses.item())

	print("------------------------->", train_loss,val_loss)

# 7. Predict

In [ ]:
model.load_state_dict(torch.load("./end2end.pth"))

In [ ]:
item=iter(val_dataloader)
images,targets=item.next()

In [ ]:
def view(images,labels,k,std=1,mean=0):
  figure=plt.figure(figsize=(30,30))
  images=list(images)
  labels=list(labels)
  for i in range(k):
    out=torchvision.utils.make_grid(images[i])
    inp=out.cpu().numpy().transpose((1,2,0))
    inp=np.array(std)*inp+np.array(mean)
    inp=np.clip(inp,0,1)  
    ax = figure.add_subplot(2,2, i + 1)
    ax.imshow(images[i].cpu().numpy().transpose((1,2,0)))
    l=labels[i]['boxes'].cpu().numpy()
    l[:,2]=l[:,2]-l[:,0]
    l[:,3]=l[:,3]-l[:,1]
    for j in range(len(l)):
      ax.add_patch(patches.Rectangle((l[j][0],l[j][1]),l[j][2],l[j][3],linewidth=2,edgecolor='w',facecolor='none')) 


In [ ]:
view(images,targets,2) # truth

In [ ]:
images = list(image.to(device) for image in images)
targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

model.eval()
model=model.double()
output=model(images)
# predict :))
with torch.no_grad():
    view(images,output,2)